<a href="https://colab.research.google.com/github/DanielaCaroline18/CompEvolucionaria--JobShopSchedulingProblem-/blob/main/ML_Naive_Bayes_clima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naïve Bayes


Ilustra o funcionamento do algoritmo de Naïve Bayes com atributos categóricos.
-------------------------------------------------------------------------------

### Base de dados: WEATHER.NOMINAL, Weka

http://storm.cis.fordham.edu/~gweiss/data-mining/datasets.html

DESCRIÇÃO DOS ATRIBUTOS:

1. Aparência {Ensolarado, Nublado, Chuvoso}
2. Temperatura {Quente, Moderado, Frio}
3. Umidade {Alta, Normal}
4. Vento {Fraco, Forte}
5. Jogar {Sim, Não}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from google.colab import files

uploaded = files.upload()

Saving clima.xlsx to clima.xlsx


### Transformação de dados

Inicialmente os atributos são convertidos para o formato de dicionário.
Em seguida os atributos são transformados em binário, para cada par (key, value) do dicionário.

<code>DictVectorizer(dtype=class ‘numpy.float64’, separator=’=’, sparse=True, sort=True)</code>

A classe é convertida para labels únicos sequenciais.

<code>
 le = preprocessing.LabelEncoder()
  
 le.fit(dados)
</code>


In [ ]:
clima_nominal = pd.read_excel("clima.xlsx", sheet_name=0) 
print("\nDimensões: {0}".format(clima_nominal.shape))
print("\nCampos: {0}".format(clima_nominal.keys()))
print(clima_nominal.describe(), sep='\n')

X_dict = clima_nominal.iloc[:,0:4].T.to_dict().values()
vect = DictVectorizer(sparse=False)
X_data = vect.fit_transform(X_dict)

le = LabelEncoder()
y_data = le.fit_transform(clima_nominal.iloc[:,4])

# Exibe o dado convertido em dicionario.
print("Atributos:", X_dict)

# Exibe a estrutura do dado convertido em binário.
print("Shape do dado de treinamento: {0}".format(X_data.shape))

print("Labels:", y_data)




Dimensões: (14, 5)

Campos: Index(['Aparência', 'Temperatura', 'Umidade', 'Vento', 'Jogar'], dtype='object')
       Aparência Temperatura Umidade  Vento Jogar
count         14          14      14     14    14
unique         3           3       2      2     2
top      Chuvoso    Moderado  Normal  Fraco   Sim
freq           5           6       7      8     9
Atributos: dict_values([{'Aparência': 'Ensolarado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Vento': 'Fraco'}, {'Aparência': 'Ensolarado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Vento': 'Forte'}, {'Aparência': 'Nublado', 'Temperatura': 'Quente', 'Umidade': 'Alta', 'Vento': 'Fraco'}, {'Aparência': 'Chuvoso', 'Temperatura': 'Moderado', 'Umidade': 'Alta', 'Vento': 'Fraco'}, {'Aparência': 'Chuvoso', 'Temperatura': 'Frio', 'Umidade': 'Normal', 'Vento': 'Fraco'}, {'Aparência': 'Chuvoso', 'Temperatura': 'Frio', 'Umidade': 'Normal', 'Vento': 'Forte'}, {'Aparência': 'Nublado', 'Temperatura': 'Frio', 'Umidade': 'Normal', 'Vento': 'F

In [ ]:
# Divide a base de dados em um conjunto de treinamento e outro de teste.
X_train, X_test, y_train, y_test = train_test_split(
                                                    X_data,
                                                    y_data,
                                                    random_state=0)

print("Dados de treinamento:\n{}".format(X_train))
print("\nClasses de treinamento:\n{}".format(y_train))

Dados de treinamento:
[[0. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 1. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 0. 1.]]

Classes de treinamento:
[1 0 1 0 0 1 1 0 0 1]


### Indução do Modelo Gaussian Naïve Bayes

O Gaussian Naïve Bayes é o algoritmo capaz de induzir um modelo a partir de dados numéricos.

Este modelo assume uma distribuição normal nos dados de treinamento.

Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ```GaussianNB()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [ ]:
from sklearn.naive_bayes import GaussianNB
nbClima = GaussianNB()
nbClima = nbClima.fit(X_train, y_train)

print("------------------Estatísticas de treinamento------------------")
print("Acurácia:", nbClima.score(X_train, y_train))

y_pred = nbClima.predict(X_train)
print("Acurácia de previsão:", accuracy_score(y_train, y_pred))
print(classification_report(y_train, 
                            y_pred, 
                            target_names=["Não", "Sim"]))

cnf_matrix = confusion_matrix(y_train, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix,
                         index=["Jogar=Não", "Jogar=Sim"],
                         columns=["Jogar(prev)=Não",
                                  "Prev. Jogar(prev)=Sim"])
print(cnf_table)

print("\n\n------------------Estatísticas de teste------------------")
y_pred_test = nbClima.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, 
                            y_pred_test, 
                            target_names=["Não", "Sim"]))

cnf_matrix_test = confusion_matrix(y_test, y_pred_test)
cnf_table_test = pd.DataFrame(data=cnf_matrix_test,
                              index=["Jogar=Não", "Jogar=Sim"],
                              columns=["Jogar(prev)=Não",
                                       "Prev. Jogar(prev)=Sim"])
print(cnf_table_test, '\n\n')

------------------Estatísticas de treinamento------------------
Acurácia: 0.9
Acurácia de previsão: 0.9
              precision    recall  f1-score   support

         Não       0.83      1.00      0.91         5
         Sim       1.00      0.80      0.89         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10

           Jogar(prev)=Não  Prev. Jogar(prev)=Sim
Jogar=Não                5                      0
Jogar=Sim                1                      4


------------------Estatísticas de teste------------------
Acurácia de previsão: 0.5
              precision    recall  f1-score   support

         Não       0.00      0.00      0.00         0
         Sim       1.00      0.50      0.67         4

    accuracy                           0.50         4
   macro avg       0.50      0.25      0.33         4
weighted avg       1.00      0.50      0.67         4

      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Indução do Modelo Multinomial Naïve Bayes

O Multinomial Naïve Bayes é o algoritmo capaz de induzir um modelo a partir de dados discretos.

Leva em consideração a frequência da ocorrência de uma palavra em um documento.

Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ```MultinomialNB()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [ ]:
from sklearn.naive_bayes import MultinomialNB
nbClima = MultinomialNB()
nbClima = nbClima.fit(X_train, y_train)

print("------------------Estatísticas de treinamento------------------")
print("Acurácia:", nbClima.score(X_train, y_train))

y_pred = nbClima.predict(X_train)
print("Acurácia de previsão:", accuracy_score(y_train, y_pred))
print(classification_report(y_train, 
                            y_pred, 
                            target_names=["Não", "Sim"]))

cnf_matrix = confusion_matrix(y_train, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix,
                         index=["Jogar=Não", "Jogar=Sim"],
                         columns=["Jogar(prev)=Não",
                                  "Prev. Jogar(prev)=Sim"])
print(cnf_table)

print("\n\n------------------Estatísticas de teste------------------")
y_pred_test = nbClima.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, 
                            y_pred_test, 
                            target_names=["Não", "Sim"]))

cnf_matrix_test = confusion_matrix(y_test, y_pred_test)
cnf_table_test = pd.DataFrame(data=cnf_matrix_test,
                              index=["Jogar=Não", "Jogar=Sim"],
                              columns=["Jogar(prev)=Não",
                                       "Prev. Jogar(prev)=Sim"])
print(cnf_table_test, '\n\n')

------------------Estatísticas de treinamento------------------
Acurácia: 0.9
Acurácia de previsão: 0.9
              precision    recall  f1-score   support

         Não       0.83      1.00      0.91         5
         Sim       1.00      0.80      0.89         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10

           Jogar(prev)=Não  Prev. Jogar(prev)=Sim
Jogar=Não                5                      0
Jogar=Sim                1                      4


------------------Estatísticas de teste------------------
Acurácia de previsão: 0.75
              precision    recall  f1-score   support

         Não       0.00      0.00      0.00         0
         Sim       1.00      0.75      0.86         4

    accuracy                           0.75         4
   macro avg       0.50      0.38      0.43         4
weighted avg       1.00      0.75      0.86         4

     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Indução do Modelo Bernoulli Naïve Bayes

O Bernoulli Naïve Bayes é o algoritmo capaz de induzir um modelo a partir de dados numéricos.

Os três passos para indução de um modelo são:

1.   Instanciar o modelo: ```BernoulliNB()```
2.   Treinar o modelo: ```fit()```
3.   Testar o modelo: ```predict()```



In [ ]:
from sklearn.naive_bayes import BernoulliNB
nbClima = BernoulliNB()
nbClima = nbClima.fit(X_train, y_train)

print("------------------Estatísticas de treinamento------------------")
print("Acurácia:", nbClima.score(X_train, y_train))

y_pred = nbClima.predict(X_train)
print("Acurácia de previsão:", accuracy_score(y_train, y_pred))
print(classification_report(y_train, 
                            y_pred, 
                            target_names=["Não", "Sim"]))

cnf_matrix = confusion_matrix(y_train, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix,
                         index=["Jogar=Não", "Jogar=Sim"],
                         columns=["Jogar(prev)=Não",
                                  "Prev. Jogar(prev)=Sim"])
print(cnf_table)

print("\n\n------------------Estatísticas de teste------------------")
y_pred_test = nbClima.predict(X_test)
print("Acurácia de previsão:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, 
                            y_pred_test, 
                            target_names=["Não", "Sim"]))

cnf_matrix_test = confusion_matrix(y_test, y_pred_test)
cnf_table_test = pd.DataFrame(data=cnf_matrix_test,
                              index=["Jogar=Não", "Jogar=Sim"],
                              columns=["Jogar(prev)=Não",
                                       "Prev. Jogar(prev)=Sim"])
print(cnf_table_test, '\n\n')

------------------Estatísticas de treinamento------------------
Acurácia: 0.9
Acurácia de previsão: 0.9
              precision    recall  f1-score   support

         Não       0.83      1.00      0.91         5
         Sim       1.00      0.80      0.89         5

    accuracy                           0.90        10
   macro avg       0.92      0.90      0.90        10
weighted avg       0.92      0.90      0.90        10

           Jogar(prev)=Não  Prev. Jogar(prev)=Sim
Jogar=Não                5                      0
Jogar=Sim                1                      4


------------------Estatísticas de teste------------------
Acurácia de previsão: 0.75
              precision    recall  f1-score   support

         Não       0.00      0.00      0.00         0
         Sim       1.00      0.75      0.86         4

    accuracy                           0.75         4
   macro avg       0.50      0.38      0.43         4
weighted avg       1.00      0.75      0.86         4

     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
